In [1]:
"""
I use this notebook to save the data from .json-file as a csv-file
"""

'\nI use this notebook to save the data from .json-file as a csv-file\n'

In [6]:
import json
import pandas as pd
from helpers import slugify, get_table_id

In [7]:
path_to_dev_tables = '/Users/bleopold/OneDrive/data-analysis/vendors/WikiSQL/data/dev.tables.jsonl'
base_target_path = "../data/01_raw/wiki-sql"

In [8]:
def get_data_for_table_as_df(result):
    table_columns = result['header']
    table_columns = [slugify(_) for _ in table_columns]
    
    table_rows = result['rows']

    table_df = pd.DataFrame(columns = table_columns)
    
    for table_row in table_rows:
        table_row_df = pd.DataFrame(columns = table_columns, data=[table_row])
        table_df = table_df.append(table_row_df)
    
    table_df.reset_index(drop=True, inplace=True)
    return table_df

# Get the data

In [9]:
with open(path_to_dev_tables, 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list[0:2]:
    result = json.loads(json_str)
    
    # get table only if page_title exists
    try:
        page_title = result["page_title"]
    except KeyError as e:
        continue
        
        
    table = get_data_for_table_as_df(result)
    
    # save table data to disk
    table_id = get_table_id(result)
    table_file_path = '{}/{}.csv'.format(base_target_path, table_id)
    print("saved table data to: {}".format(table_file_path))
    table.to_csv(table_file_path, sep='\t', index=False)

saved table data to: ../data/01_raw/wiki-sql/1-10015132-11.csv
saved table data to: ../data/01_raw/wiki-sql/1-10015132-9.csv
